# Inizializzazione

In [5]:
import matplotlib.pyplot as plt

In [6]:
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

In [7]:
# data = pd.read_csv("data\compas-analysis\compas-scores.csv")

In [8]:
print(sklearn.__version__)

1.5.2


# Ricerca di colonne ridondanti

In [ ]:
def eq_col(data, feat1, feat2):
    for x in range(0, 11757):
        assert data[feat1].iloc[x] == data[feat2].iloc[x]
    print(f"{feat1} è uguale a {feat2}")

Da questa funzione risultano uguali le colonne:
* "decile_score" e "decile_score.1"
* "compas_screening_date", "v_screening_date" e "screening_date"

In [ ]:
def duplicati(data, feature):
    n_arr = data[feature]
    n_set = {x for x in n_arr}
    return n_set

def display(data):
    print("-----------")
    for item in data.columns:
        dup = duplicati(data, item)
        if item == "id":
            tot = len(dup)
        if item != "id":
            res = len(dup)
            if tot == res:
                print(item, "CANDIDATE KEY!")
            else:
                if res > 20:
                    print(item, res)
                else:
                    print(item, dup)

Da questa funzione risultano:

* In quanto ridondantemente categorici:

    * age_cat
    * v_score_text
    * score_text

* In quanto poco informativi:

    * num_r_cases
    * num_vr_cases
    * v_type_of_assessment = {'Risk of Violence'} 
    * type_of_assessment = {'Risk of Recidivism'}


In [ ]:
data = pd.DataFrame(data=data, columns=[
    'id',
    'name',
    'first',
    'last',
    'sex',
    'dob',
    'age',
    'race',
    'juv_fel_count',
    'juv_misd_count',
    'juv_other_count',
    'priors_count',
    'days_b_screening_arrest',
    'c_jail_in',
    'c_jail_out',
    'c_case_number',
    'c_offense_date',
    'c_arrest_date',
    'c_days_from_compas',
    'c_charge_degree',
    'c_charge_desc',
    'is_recid',
    'r_case_number',
    'r_charge_degree',
    'r_days_from_arrest',
    'r_offense_date',
    'r_charge_desc',
    'r_jail_in',
    'r_jail_out',
    'is_violent_recid',
    'vr_case_number',
    'vr_charge_degree',
    'vr_offense_date',
    'vr_charge_desc',
    'v_decile_score',
    'decile_score',
    'screening_date'
])

E' interessante notare che possiamo estrarre la storia criminale dei soggetti registrati nel database, con la seguente funzione:

In [ ]:
history = pd.DataFrame(data=data, columns=[
    'id',
    'dob',
    'c_jail_in',
    'c_jail_out',
    'c_offense_date',
    'r_offense_date',
    'r_jail_in',
    'r_jail_out',
    'vr_offense_date',
    'screening_date'
])

In [ ]:
# display(data)
print(data['dob'].unique())


# Preprocessing

La tabella contiene dei valori non conformi, come notiamo applicando la funzione "display()" alla nuova tabella "data":
* La colonna "is_recid" ha valori -1.

In [ ]:
data = data[data.is_recid.isin([0.0, 1.0])]
data = data[data.v_decile_score.isin(range(1, 11))]
data = data[data.decile_score.isin(range(1, 11))]
print(data)

* Alcune colonne contengono dati categorici.

In [ ]:
columns_to_encode = ['sex', 'race', 'c_charge_degree', 'r_charge_degree', 'vr_charge_degree']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')  
encoded_data = encoder.fit_transform(data[columns_to_encode])
encoded_columns = encoder.get_feature_names_out(columns_to_encode) # Nomi delle nuove colonne
encoded_df = pd.DataFrame(encoded_data, columns=encoded_columns)
data = data.drop(columns=columns_to_encode)  # Rimuovere le colonne originali
data = pd.concat([data, encoded_df], axis=1)  # Aggiungere le nuove colonne codificate
print(data)

Per la gestione delle date:
* Le date di incarcerazione sono convertite nel numero di giorni vissuti da innocente
* Le date di scarcerazione sono convertite nel numero di giorni vissuti in carcere

Piccola nota metodologica:

$ d_2 - d_1 = d_2 + (d_0 - d_0) - d_1 = (d_2 - d_0) - (d_1 - d_0) $

Questo espediente richiede, come minimo, che la struttura algebrica di riferimento sia almeno un gruppo (additivo). Le date costituiscono un gruppo (additivo, nonfinito), poiché è banale l'isomorfismo tra questa struttura algebrica e un gruppo costituito dall'insieme dei numeri interi e l'operazione di incremento.

Premesso, dunque, che $d_0$ può essere una data qualsiasi, tre scelte:

* La scelta convenzionale: $d_0 = 01/01/1970$;
* La scelta più logica: $d_0 = min(date \_ nel \_ dataset) = 14/10/1919$;
* La scelta effettivamente selezionata: $d_0 = 24/6/1936$ (è una data di interesse per lo sviluppatore, ed è anche sufficientemente anteriore da coprire tutte le date nel dataset, tranne 5 date di nascita nella colonna "dob")

In [16]:
from datetime import date
import re

is_valid_date = lambda value: isinstance(value, str) and bool(re.match(r'^\d{4}-\d{2}-\d{2}$', value))
data = data[data['dob'].apply(is_valid_date)]

print(data)

pot_dates = lambda giorno: (giorno-date(1936,6,24)).days
date_converter = lambda item: pot_dates(date(*[int(n) for n in item.split('-')]))
col_converter = lambda data, col: [date_converter(data.loc[i, col]) for i in range(len(data[col]))]

test = data.copy()

item = data['dob'][0]
print(item, item.__class__)
trans = date_converter(item)
print(trans, trans.__class__)

item_test = test.loc[0, 'dob']
print(item_test, item_test.__class__)
trans_test = date_converter(item_test)
print(trans_test, trans_test.__class__)

def date_converter2(item):
    res = item.split('-')
    res2 = [int(n) for n in res]
    res3 = date(*res2)
    res4 = pot_dates(res3)
    return res4

try:
    test['dob'] = col_converter(test, 'dob')
except AttributeError as e:
    print(e)
    print(e.obj)

print(date_converter(data['screening_date'][0]))

            id                    name     first           last         dob  \
0          1.0        miguel hernandez    miguel      hernandez  1947-04-18   
2          3.0             kevon dixon     kevon          dixon  1982-01-22   
3          4.0                ed philo        ed          philo  1991-05-14   
4          5.0             marcu brown     marcu          brown  1993-01-21   
5          6.0      bouthy pierrelouis    bouthy    pierrelouis  1973-01-22   
...        ...                     ...       ...            ...         ...   
11752  11753.0        patrick hamilton   patrick       hamilton  1968-05-02   
11753  11754.0       raymond hernandez   raymond      hernandez  1993-06-24   
11754  11755.0  dieuseul pierre-gilles  dieuseul  pierre-gilles  1981-01-24   
11755  11756.0        scott lomagistro     scott     lomagistro  1986-12-04   
11756  11757.0                chin yan      chin            yan  1982-02-19   

        age  juv_fel_count  juv_misd_count  juv_oth

KeyError: 1

In [ ]:
import pandas as pd
from datetime import datetime

# Funzione per calcolare la differenza in giorni da una data di riferimento
def calcola_differenza(data, riferimento):
    try:
        # Converte il valore in una data; se fallisce, restituisce il valore originale
        data_converted = pd.to_datetime(data, format='%Y-%m-%d', errors='coerce')
        if pd.isnull(data_converted):
            return data  # Restituisce il valore originale se non è una data valida
        return (data_converted - riferimento).days
    except Exception as e:
        return data  # Restituisce il valore originale in caso di errore

# Data di riferimento
data_riferimento = datetime(1936, 6, 24)

# Lettura del dataset CSV
file_csv = r"C:\Users\gianluca.colasuonno\compas-scores.csv"
df = pd.read_csv(file_csv)

# Identificazione e trasformazione delle colonne con date
for col in df.columns:
    if df[col].dtype == 'object':  # Verifica se la colonna contiene stringhe
        # Applica la funzione di calcolo differenza alle colonne candidate
        df[col] = df[col].apply(lambda x: calcola_differenza(x, data_riferimento))

# Stampa il risultato
print("\nDataFrame trasformato:")
print(df)

In [19]:
diff_dates = lambda date2, date1: abs(date2-date1).days
pot_dates = lambda giorno: (giorno-date(1936,6,24)).days
pot2_dates = lambda date2, date1: pot_dates(date2) - pot_dates(date1)

def main():
    d1 = date(2000,2,28)
    d2 = date(2013,9,13)
    result1 = diff_dates(d2, d1)
    result2 = pot2_dates(d2, d1)
    print('{} days between {} and {}'.format(result1, d1, d2))
    print('{} days between {} and {}'.format(result2, d1, d2))
    print("Happy programmer's day!")

# main()

career = ['dob', 'screening_date', 'c_offense_date', 'c_jail_in', 'c_jail_out','r_offense_date', 'r_jail_in', 'r_jail_out', 'vr_offense_date']

print(pot_dates(date(2013,8,14)))
print(pot_dates(date(2014,12,31)))

28175
28679


* Alcune colonne contengono dati numerici, ma che devono essere riscalati, pena la confusione del modello sull'importanza di certe features contro l'importanza di altre.

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler, QuantileTransformer

# 1 Istanzi oggetto per il Min-Max scaling

scaler = MinMaxScaler()

# 2 Applichi lo scaling sui dati numerici del DataFrame

df_minmax_scaled = data.copy() # Fai una copia per non sovrascrivere il DataFrame originale

# 3 Selezioni colonne numeriche (non includo le colonne codificate One Hot)

numerical_columns = df_minmax_scaled.select_dtypes(include=['float64', 'int64']).columns

# 4 Applichi lo scaling

df_minmax_scaled[numerical_columns] = scaler.fit_transform(df_minmax_scaled[numerical_columns])

print(df_minmax_scaled)

# Modelli

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
from sklearn.metrics import precision_score, recall_score, make_scorer
names = ['name', 'first', 'last']
career = ['dob', 'screening_date', 'c_offense_date', 'c_jail_in', 'c_jail_out','r_offense_date', 'r_jail_in', 'r_jail_out', 'vr_offense_date']
target = ['decile_score']
X = df_minmax_scaled.filter(items=target, axis=1).values
y = df_minmax_scaled['decile_score'].values

f"Shape of X={X.shape} /n Shape of y={y.shape}"

mod1 = LogisticRegression()
mod1.fit(X, y)
mod1.predict(X)

# Metrica personalizzata
def min_rec_prec(y_true, y_pred):
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    return min(recall, precision)

def scored_min_rec_prec(est, X, y_true, sample_weight=None):
    y_pred = est.predict(X)
    return min_rec_prec(y_true, y_pred)

grid = GridSearchCV(
    estimator = LogisticRegression,
    # param_grid da inserire: tra quali delle configurazioni che indico devo far ottimizzare il modello, date le metriche?
    cv = 4,
    scoring = {'precision': make_scorer(precision_score),
               'recall': make_scorer(recall_score),
               'min_rec_prec': make_scorer(min_rec_prec)},
    refit='min_rec_prec', # refit: quale delle metriche deve essere utilizzata dal modello per ottimizzarsi?

)
grid.fit(X, y)

plt.figure(figsize=(12, 4))
results = pd.DataFrame(grid.cv_results_) # la metrica è nelle impostazioni dello stimatore selezionato: di default è l'accuracy score = TP+TN/P+N ("vero su totale")
for score in ['mean_test_recall', 'mean_test_precision', 'mean_test_min_rec_prec']:
    plt.plot([_[1] for _ in results['param_class_weight']], results[score], label=score)
plt.legend()

